# Анализ результатов эксперимента от 23.09.25

In [ ]:
%load_ext autoreload
%autoreload 2

## Описание эксперимента

Проверялась работа алгоритма при отправке фиксированных значений Kp = 3.5, Ki = 11.0, Kd = 0.002.
Также была внедрена отправка control min и control max.

## Результаты

In [ ]:
from pathlib import Path

EXPERIMENT_NAME = "connection_test"
EXPERIMENT_DATE = "2025-09-23"
EXPERIMENT_TIME = "17-44-34"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}/{EXPERIMENT_TIME}")

LOG_TRAIN_DIR = PATH_TO_EXP_DIR / "train_logs"
LOG_ENV_DIR = PATH_TO_EXP_DIR / "env_logs"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_tensorboard_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

In [ ]:
from nn_laser_stabilizer.logging.utils import tensorboard_to_df

train_df = tensorboard_to_df(str(LOG_TRAIN_DIR))

Tensorboard содержит не все данные: количество step значительно количества точек.

In [ ]:
print(train_df.info())
print(train_df["step"])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, train_df, tags=["Loss/Critic", "Loss/Actor"], title="Training curves")
plt.tight_layout()
plt.savefig(LOG_TRAIN_DIR / "train_logs.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(left=0, right=500)
plot_tensorboard_df(ax, train_df, tags=["Loss/Critic", "Loss/Actor"], title="Training curves")
plt.tight_layout()
plt.savefig(LOG_TRAIN_DIR / "train_logs_first_steps.pdf")

In [ ]:
import pandas as pd
import re

def parse_pid_log(file_path: str) -> pd.DataFrame:
    pattern = re.compile(
        r"step=(\d+)\s+time=([-+]?\d*\.\d+|\d+)\s+kp=([-+]?\d*\.\d+|\d+)\s+"
        r"ki=([-+]?\d*\.\d+|\d+)\s+kd=([-+]?\d*\.\d+|\d+)\s+"
        r"x=([-+]?\d*\.\d+|\d+)\s+control_output=([-+]?\d*\.\d+|\d+)\s+"
        r"setpoint=([-+]?\d*\.\d+|\d+)"
    )

    data = []
    with open(file_path, "r") as f:
        for line in f:
            match = pattern.search(line)
            if match:
                data.append([
                    int(match.group(1)),      # step
                    float(match.group(2)),    # time
                    float(match.group(3)),    # kp
                    float(match.group(4)),    # ki
                    float(match.group(5)),    # kd
                    float(match.group(6)),    # x
                    float(match.group(7)),    # control_output
                    float(match.group(8))     # setpoint
                ])

    df = pd.DataFrame(data, columns=[
        "step", "time", "Action/kp", "Action/ki", "Action/kd", 
        "Observation/x", "Observation/control_output", "Observation/setpoint"
    ])
    df["reward"] = -abs(df["Observation/setpoint"] - df["Observation/x"]) + 1
    return df

In [ ]:
env_df = parse_pid_log(str(LOG_ENV_DIR / "log.txt"))

In [ ]:
# Делаем обратное преобразование для графика
env_df["Observation/x"] = env_df["Observation/x"].apply(lambda x : ((x + 1.0) / 2) * 10230)
env_df["Observation/setpoint"] = env_df["Observation/setpoint"].apply(lambda x : ((x + 1.0) / 2) * 10230)

env_df["Observation/control_output"] = env_df["Observation/control_output"].apply(lambda x : ((x + 1.0) / 2) * 10230)

In [ ]:
env_df["time_sec"] = env_df["time"] - env_df["time"].min()

## График для ВКВО

In [ ]:
plt.rcParams.update({
    'font.size': 14,          # шрифт для подписей осей и текста
    'axes.labelsize': 16,     # шрифт для подписей осей
    'axes.titlesize': 18,     # шрифт заголовка графика
    'xtick.labelsize': 14,    # шрифт делений по X
    'ytick.labelsize': 14,    # шрифт делений по Y
    'legend.fontsize': 14     # шрифт легенды
})

SETPOINT = 1200

fig, ax1 = plt.subplots(figsize=(10, 5))

ax1.plot(env_df["time_sec"], env_df["Observation/x"], color="green", label=r"$U_{\text{АЦП}}$")
ax1.axhline(SETPOINT, color="black", linestyle="--")
ax1.set_xlabel("Время (с)")
ax1.set_ylabel("Напряжение на фотодетекторе (АЦП)", color="green")
ax1.tick_params(axis='y', labelcolor="green")
ax1.set_ylim(950, 1850)
ax1.set_xlim(1, 2000)

ax2 = ax1.twinx()
ax2.plot(env_df["time_sec"], env_df["Observation/control_output"], color="red", label=r"$U_{\text{ЦАП}}$")
ax2.set_ylabel("Напряжение на фазовращателе (ЦАП)", color="red")
ax2.tick_params(axis='y', labelcolor="red")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="upper right")

plt.tight_layout()
plt.savefig("work.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint", "Observation/control_output"], title="Observation curves")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "observation_logs.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(left=200_000, right=200_500)
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint", "Observation/control_output"], title="Observation curves")
# ax.set_ylim(500, 2050)
fig.savefig(LOG_ENV_DIR / "observation_logs_steps.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["reward"], title="Reward curve")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "reward_logs.pdf")

In [ ]:
action_tags = ["Action/kp", "Action/ki", "Action/kd"]

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, 1000)

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    ax.axhline(y=0)

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}_first_steps.pdf")

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}.pdf")

In [ ]:
from datetime import timedelta

env_df["delta_ms"] = env_df["time"].diff() * 1000
print(env_df["delta_ms"].describe())

total_seconds = env_df["time"].iloc[-1] - env_df["time"].iloc[0]

td = timedelta(seconds=total_seconds)
print(f"Полное время: {td}")

## Исследование

In [ ]:
start = 200_000
window_size = 50

window = env_df.iloc[start:start+window_size]

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(left=start, right=start + window_size)
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint"], title="Observation curves")

stats = window["Observation/x"].agg(["mean", "std", "min", "max"])
print(stats)

window_sizes = [n for n in range(1, 51)]

means = []
for ws in window_sizes:
    window = env_df.iloc[start:start+ws]
    means.append(window["Observation/x"].mean() - 1200)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(window_sizes, means, marker="o")
ax.set_xlabel("Window size")
ax.set_ylabel("Mean of Observation/x")
ax.set_title(f"Mean vs Window size (start={start})")
plt.show()

In [ ]:
start = 450_000
window_size = 50

window = env_df.iloc[start:start+window_size]

fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(left=start, right=start + window_size)
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint"], title="Observation curves")

stats = window["Observation/x"].agg(["mean", "std", "min", "max"])
print(stats)

window_sizes = [n for n in range(1, 51)]

means = []
for ws in window_sizes:
    window = env_df.iloc[start:start+ws]
    means.append(window["Observation/x"].mean() - 1200)

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(window_sizes, means, marker="o")
ax.set_xlabel("Window size")
ax.set_ylabel("Mean of Observation/x")
ax.set_title(f"Mean vs Window size (start={start})")
plt.show()